In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import time
from sqlalchemy import create_engine

crime_date_load = "data/Crime_Incidents_in_2019.csv"
crime_date = pd.read_csv(crime_date_load)

crime_data_head()

FileNotFoundError: [Errno 2] File b'data/Crime_Incidents_in_2019.csv' does not exist: b'data/Crime_Incidents_in_2019.csv'

In [2]:
crime_date["START_DATE"] = crime_date.START_DATE.str.split('T').str[0]
crime_date_df = crime_date[[ "CCN","SHIFT","OFFENSE", "LATITUDE","LONGITUDE", "START_DATE", "WARD"]]
crime_date_df.head()

,CCN,SHIFT,OFFENSE,LATITUDE,LONGITUDE,START_DATE,WARD
0,10199597,DAY,THEFT/OTHER,38.887592,-76.982941,2019-11-07,6
1,19195892,EVENING,THEFT F/AUTO,38.915020,-77.027039,2019-11-01,1
2,19195912,EVENING,THEFT/OTHER,38.912454,-77.021919,2019-10-31,6
3,19195924,EVENING,MOTOR VEHICLE THEFT,38.982384,-77.026557,2019-11-01,4
4,19195933,EVENING,THEFT/OTHER,38.909103,-77.021918,2019-11-01,6


In [3]:
crime_date_df.dtypes

CCN             int64
SHIFT          object
OFFENSE        object
LATITUDE      float64
LONGITUDE     float64
START_DATE     object
WARD            int64
dtype: object

In [ ]:
# connection_string = "postgres:postgres@localhost:5432/dc_crime"
# engine = create_engine(f'postgresql://{connection_string}')
# engine.table_names()

In [ ]:
# crime_date_df.to_sql(name='crime_date_df', con=engine, if_exists='append', index=False)
# pd.read_sql_query('select * from crime_date_df', con=engine).head()

In [4]:
start_date_df = crime_date[["START_DATE", "WARD"]]
start_date_df.set_axis(["day", "value"],axis='columns', inplace=True)

date_count=start_date_df.groupby(["day"]).count()
date_count.head()
# date_count.to_json(r'\Users\Owner\Desktop\Project-2\data\date_count.json', orient='table')
# date_count.to_csv(r'\Users\Owner\Desktop\Project-2\data\date_count.csv')

,value
day,
2005-04-17,1
2012-12-04,1
2016-05-10,1
2016-10-20,1
2016-10-28,1


In [5]:
shift_df = crime_date[[ "SHIFT", "WARD"]]
shift_count=shift_df.groupby(["SHIFT"]).count()
print(shift_count)


           WARD
SHIFT          
DAY       11460
EVENING   14927
MIDNIGHT   7539


In [12]:
ward_df = crime_date[["OFFENSE", "WARD"]]
ward_df.set_axis(["OFFENSE", "WARD"],axis='columns', inplace=True)

ward_one = ward_df[(ward_df["OFFENSE"] == "ARSON")]
ward_one_group=ward_one.groupby(["WARD"]).count()
ward_one_group.set_axis(["Arson"],axis='columns', inplace=True)

ward_two = ward_df[(ward_df["OFFENSE"] == "ASSAULT W/DANGEROUS WEAPON")]
ward_two_group=ward_two.groupby(["WARD"]).count()
ward_two_group.set_axis(["Assault w/ Dangerous Weapon"],axis='columns', inplace=True)

ward_three = ward_df[(ward_df["OFFENSE"] == "BURGLARY")]
ward_three_group=ward_three.groupby(["WARD"]).count()
ward_three_group.set_axis(["Burglary"],axis='columns', inplace=True)

ward_four = ward_df[(ward_df["OFFENSE"] == "HOMICIDE")]
ward_four_group=ward_four.groupby(["WARD"]).count()
ward_four_group.set_axis(["Homicide"],axis='columns', inplace=True)

ward_five = ward_df[(ward_df["OFFENSE"] == "MOTOR VEHICLE THEFT")]
ward_five_group=ward_five.groupby(["WARD"]).count()
ward_five_group.set_axis(["Motor Vehicale Theft"],axis='columns', inplace=True)

ward_six = ward_df[(ward_df["OFFENSE"] == "ROBBERY")]
ward_six_group=ward_six.groupby(["WARD"]).count()
ward_six_group.set_axis(["Robbery"],axis='columns', inplace=True)

ward_seven = ward_df[(ward_df["OFFENSE"] == "SEX ABUSE")]
ward_seven_group=ward_seven.groupby(["WARD"]).count()
ward_seven_group.set_axis(["Sex Abuse"],axis='columns', inplace=True)

ward_eight = ward_df[(ward_df["OFFENSE"] == "THEFT F/AUTO")]
ward_eight_group=ward_eight.groupby(["WARD"]).count()
ward_eight_group.set_axis(["Theft f/ Auto"],axis='columns', inplace=True)

ward_nine = ward_df[(ward_df["OFFENSE"] == "THEFT/OTHER")]
ward_nine_group=ward_nine.groupby(["WARD"]).count()
ward_nine_group.set_axis(["Theft / other"],axis='columns', inplace=True)



,Motor Vehicale Theft
WARD,
1,250
2,260
3,96
4,207
5,388
6,286
7,465
8,271


In [16]:
join_one=ward_three_group.join(ward_five_group)
join_two=join_one.join(ward_one_group)
join_three=join_two.join(ward_two_group)
join_four=join_three.join(ward_six_group)
join_five=join_four.join(ward_four_group)
join_six=join_five.join(ward_seven_group)
join_seven=join_six.join(ward_eight_group)
final_join=join_seven.join(ward_nine_group)
Ward_counts_offense=final_join.fillna(0)
Ward_counts_offense.head(8)


,Burglary,Motor Vehicale Theft,Arson,Assault w/ Dangerous Weapon,Robbery,Homicide,Sex Abuse,Theft f/ Auto,Theft / other
WARD,,,,,,,,,
1,130,250,1.0,163,401,15.0,17,1644,2113
2,197,260,1.0,137,256,0.0,28,2230,4008
3,86,96,0.0,24,52,3.0,10,628,1058
4,123,207,0.0,69,165,5.0,12,1087,1228
5,245,388,1.0,274,349,21.0,24,1753,2027
6,141,286,2.0,174,353,16.0,26,1889,2755
7,161,465,0.0,369,373,42.0,34,865,1428
8,190,271,3.0,357,288,63.0,37,610,942


In [27]:
# Ward_counts_offense.to_json(r'\Users\Owner\Desktop\Project-2\data\ward_offense_count.json')
date_count.to_csv(r'\Users\Owner\Desktop\Project-2\data\ward_offense_count.csv')

In [25]:
OFFENSE_df = crime_date[[ "OFFENSE", "WARD"]]
OFFENSE_count=OFFENSE_df.groupby(["OFFENSE"]).count()
print(OFFENSE_count)

                             WARD
OFFENSE                          
ARSON                           8
ASSAULT W/DANGEROUS WEAPON   1567
BURGLARY                     1273
HOMICIDE                      165
MOTOR VEHICLE THEFT          2223
ROBBERY                      2237
SEX ABUSE                     188
THEFT F/AUTO                10706
THEFT/OTHER                 15559
